In [1]:
'jai guru dev'
'om gam ganapataye namah'

'om gam ganapataye namah'

In [2]:
'git commit via bash shell script'
import requests
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_rows', 500)

In [3]:
def intraday2(stock,interval='1m',rangee='1d'):
#     stock='ICICIBANK';interval='1m';rangee='1d'
    url = f'https://query1.finance.yahoo.com/v7/finance/chart/{stock}.NS?&interval={interval}&range={rangee}'
    ab = requests.get(url)
    asd = ab.json()
    asd_list = asd.get('chart').get('result')
    sto_dict = asd_list[0].get('indicators').get('quote')[0]
    df = pd.DataFrame.from_dict(sto_dict)
    df['timestamp'] = asd_list[0].get('timestamp')
    df['timestamp'] = df['timestamp'].apply(lambda x:datetime.fromtimestamp(x))
    df = df.set_index('timestamp')
    df = df[['open','high','low','close','volume']]
    return df

In [80]:
df2 = intraday2('MAXHEALTH','1m','7d')

In [58]:
def df_process(df)   :
    df['VA'] = (df['volume'].shift(1).rolling(window=5).mean())
    # df['PA'] = (df['close'].ewm(span=5).mean()).round(2)
    df['PA'] = (df['close'].shift(1).rolling(window=5).mean())
    df['VR'] = (df['volume']/df['VA'])
    df['PR'] = ((df['close']/df['PA']-1)*10000)
    df['VA2'] = (df['VR'].shift(1).rolling(window=5).mean())
    df['PA2'] = (df['PR'].shift(1).rolling(window=5).mean())
    df['VR2'] = (df['VR']/df['VA2'])
    df['PR2'] = ((df['PR']/df['PA2']))
    df['signal'] = ['signal' if vr > 1.5 and (vr2>1.5 or vr2<0.5) and (pr2<-5 or pr2 >5) else '-' for vr,vr2,pr2 in zip(df['VR'],df['VR2'],df['PR2']) ]
    return df

In [60]:
df = df_process(df2)

In [ ]:
df.round(2)

In [20]:
dfg = df2.groupby(pd.Grouper(level='timestamp',freq='D'))

In [81]:
# dfg = df2.groupby(df2.index)
df2['date'] = [a.date() for a in df2.index]
x = df2.groupby(['date'],as_index=False)
fgh = list(x)

In [82]:
proc_list = [df_process(d[1]) for i,d in enumerate(fgh)]

In [83]:
proc_list[1].round(2)

,open,high,low,close,volume,date,VA,PA,VR,PR,VA2,PA2,VR2,PR2,signal
timestamp,,,,,,,,,,,,,,,
2021-06-04 09:15:00,246.75,247.00,246.45,246.65,0.0,2021-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
2021-06-04 09:16:00,246.95,246.95,245.25,246.20,68375.0,2021-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
2021-06-04 09:17:00,246.10,246.10,245.50,245.80,37553.0,2021-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
2021-06-04 09:18:00,245.50,246.35,245.10,246.35,28839.0,2021-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
2021-06-04 09:19:00,245.50,246.60,245.50,246.60,30954.0,2021-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-
2021-06-04 09:20:00,246.70,248.80,246.70,248.45,74139.0,2021-06-04,33144.2,246.32,2.24,86.47,NaN,NaN,NaN,NaN,-
2021-06-04 09:21:00,248.50,249.00,248.50,249.00,91851.0,2021-06-04,47972.0,246.68,1.91,94.05,NaN,NaN,NaN,NaN,-
2021-06-04 09:22:00,248.80,249.75,248.80,249.20,117341.0,2021-06-04,52667.2,247.24,2.23,79.27,NaN,NaN,NaN,NaN,-
2021-06-04 09:23:00,248.35,249.50,248.35,249.50,95890.0,2021-06-04,68624.8,247.92,1.40,63.73,NaN,NaN,NaN,NaN,-
